## AWS Serverless Lab - Stock Info
Sign up for an IEX key here:
https://iexcloud.io/

You'll need to verify your email address (use plus (+) addressing!)

Two sample API calls we'll be using:

`https://cloud.iexapis.com/stable/account/metadata?token=<TOKEN>&format=json`  
`https://cloud.iexapis.com/stable/stock/AAPL/quote?token=<TOKEN>&format=json`


## Lambda Handler

In [ ]:
import json
import logging
import os
from datetime import datetime
from urllib.parse import urljoin

import boto3
try:
    import requests
except ImportError:
    # for AWS Lambda
    from botocore.vendored import requests


logger = logging.getLogger()
logger.setLevel(logging.INFO)

IEX_KEY = os.getenv('IEX_KEY')
S3_EXTRACT_BUCKET = os.getenv('S3_EXTRACT_BUCKET')
msg = "env var IEX_KEYS & S3_EXTRACT_BUCKET must be set"
assert IEX_KEY and S3_EXTRACT_BUCKET, msg

STOCK_LIST = [
    'AAPL',
    'MSFT',
    'AMZN',
    'GOOG',
    'GOOGL'
]


class IEXApi:
    def __init__(self):
        self.base_url = 'https://cloud.iexapis.com/stable/'
        self.params = {
            'token': IEX_KEY,
            'format': 'json'
        }

    @property
    def endpoint(self):
        raise NotImplementedError

    def get_metadata(self):
        url = urljoin(self.base_url, '/account/metadata')
        resp = requests.get(url, params=self.params)
        if resp.status_code == 200:
            return resp.json()
        else:
            logger.error(f"problem with request to {url}")
            logger.info(f"params: {self.params}")
            logger.info(resp.content)


class IEXStockQuote(IEXApi):
    def __init__(self, symbol):
        super(IEXStockQuote, self).__init__()
        self.symbol = symbol
        self._endpoint = f'stock/{self.symbol}/quote'

    @property
    def endpoint(self):
        return self._endpoint

    def get_quote(self):
        url = urljoin(self.base_url, self.endpoint)
        logger.info(f"fetching url {url}")
        resp = requests.get(url, params=self.params)
        if resp.status_code == 200:
            return resp.json()
        else:
            logger.error(f"problem with request to {url}")
            logger.info(f"params: {self.params}")
            logger.info(resp.content)


def s3_upload(obj, bucket, object_name=None):

    s3 = boto3.resource('s3')
    if not object_name:
        object_name = datetime.now().strftime('%Y/%m/%d/%H/%M.%S.%f.json')
    return s3.Object(bucket, object_name).put(Body=obj)


def extract_handler(event, context):
    logger.info('## ENVIRONMENT VARIABLES')
    logger.info(os.environ)
    logger.info('## EVENT')
    logger.info(event)

    quotes = []

    for symbol in STOCK_LIST:
        stock = IEXStockQuote(symbol)
        quotes.append(stock.get_quote())

    # AWS Athena wants JSON in JSONL format
    jsonl = '\n'.join([json.dumps(quote) for quote in quotes])

    resp = s3_upload(jsonl, S3_EXTRACT_BUCKET)
    if resp['ResponseMetadata'].get('HTTPStatusCode') == 200:
        return {
            'statusCode': 200,
            'body': json.dumps('Success!')
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps(f'error! {str(resp)}')
        }


## S3 Permission Statement
```json
          {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": [
              "arn:aws:s3:::ae-demo1-extract",
              "arn:aws:s3:::ae-demo1-extract/*"
            ]
          }
```

## Athena Create Table Statement
```sql
create database demo1;

CREATE EXTERNAL TABLE iex_stocks_json (
   avgTotalVolume bigint,
   calculationPrice string,
   change float,
   changePercent float,
   close float,
   closeTime bigint,
   companyName string,
   delayedPrice float,
   delayedPriceTime bigint,
   extendedChange float,
   extendedChangePercent float,
   extendedPrice float,
   extendedPriceTime bigint,
   high float,
   iexAskPrice bigint,
   iexAskSize bigint,
   iexBidPrice bigint,
   iexBidSize bigint,
   iexLastUpdated bigint,
   iexMarketPercent float,
   iexRealtimePrice float,
   iexRealtimeSize bigint,
   iexVolume bigint,
   lastTradeTime bigint,
   latestPrice float,
   latestSource string,
   latestTime string,
   latestUpdate bigint,
   latestVolume bigint,
   low float,
   marketCap bigint,
   open float,
   openTime bigint,
   peRatio float,
   previousClose float,
   symbol string,
   week52High float,
   week52Low float,
   ytdChange float
)   
ROW FORMAT  serde 'org.apache.hive.hcatalog.data.JsonSerDe'
LOCATION 's3://YOUR-BUCKET-HERE/';
```